## Analyzing BGP Route Policies

Route policies for BGP are complex and error prone.  In this notebook we show how Batfish can be used to validate that your route policies behave as intended.  Batfish's `testRoutePolicies` question provides easy exploration of route-policy behavior through testing, while its `searchRoutePolicies` question actively searches for errors in a route policy by analyzing *all possible* behaviors.

To illustrate these capabilities, we'll use an example network with two border routers, named `border1` and `border2`, which each have a BGP session with a router of another AS.  Our goal in this notebook is to validate important properties of the in-bound and out-bound route policies for these sessions, respectively called `from_outside` and `to_outside`.

Let's start, as usual, by initializing the example network that we will use in this notebook.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/validating-route-policies?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py
load_questions()

# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/route-analysis"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

### Example 1: Filter private addresses in-bound


First, we'd like to ensure that the two `from_outside` route policies properly filter announcements for private addresses.  Traditionally you might validate this policy through some form of in-network testing.  With Batfish's `testRoutePolicies` question we can test a route policy's behavior without access to the live network.

In [2]:
inRoute1 = BgpRoute(network="10.0.0.0/24", originatorIp="5.5.5.5", originType="egp", protocol="bgp")
bfq.testRoutePolicies(policies="from_outside", direction="in", inputRoutes=[inRoute1]).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border1,from_outside,"BgpRoute(network='10.0.0.0/24', originatorIp='5.5.5.5', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",DENY,None,None
1,border2,from_outside,"BgpRoute(network='10.0.0.0/24', originatorIp='5.5.5.5', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",DENY,None,None


The first line of code above creates a `BgpRoute` object that specifies the input route announcement to use for testing, which in this case announces the prefix 10.0.0.0/24, has an originator IP that we arbitrarily chose, and has default values for other parts of the announcement.  The second line uses `testRoutePolicies` to test the behavior of the two `from_outside` route policies on this announcement.  The required `direction` parameter indicates that these are in-bound route policies.

The output of the question shows the results of the test.  In both border routers, the `from_outside` route policy properly denies this private address.

That result gives us some confidence in our route policies, but it is just a single test.  Of course we can run `testRoutePolicies` more times to test other private addresses.  However, we'd really like to know that *all* private addresses will be denied by the two in-bound route maps.  To do that, we will use the `searchRoutePolicies` question and change our perspective a bit.  Instead of providing particular test announcements and seeing how they behave, we will ask Batfish to search for a route-policy behavior that violates our intent.  If we get one or more results, then we've found a bug.  If we get no results, then we can be sure that our configurations satisfy the intent, since Batfish explores *all possible* route-policy behaviors.

In [3]:
privateIps = ["10.0.0.0/8:8-32", "172.16.0.0/28:28-32", "192.168.0.0/16:16-32"]
inRoutes1 = BgpRouteConstraints(prefix=privateIps)
bfq.searchRoutePolicies(policies="from_outside", inputConstraints=inRoutes1, action="permit").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border2,from_outside,"BgpRoute(network='192.168.0.0/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[{'asns': [44], 'confederation': False}], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='192.168.0.0/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[{'asns': [44], 'confederation': False}], communities=['2:30'], localPreference=0, metric=0, sourceProtocol=None)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[2:30]')])"


The first line above specifies the space of all private IP prefixes.  The second line creates a `BgpRouteConstraints` object, which is like the `BgpRoute` object we saw earlier but represents a set of announcements rather than a single one.  In this case, we are interested in all announcements that announce a prefix in `privateIps`.  Finally, the third line of code uses `searchRoutePolicies` to search for an announcement in the set `inRoutes` that is permitted by the `from_outside` route policy.

There are no results for `border1`, which means that its `from_outside` route policy properly filters *all* private addresses. However, the result for `border2` shows that its version of `from_outside` permits an announcement for the prefix 192.168.0.0/32.  The table also shows the route announcement that will be produced by `from_outside` in this case, along with a "diff" of the input and output announcements. Inspecting the configurations, we see that both route policies deny all announcements for prefixes in the prefix list `private-ips`, but the definitions of this prefix list differ in one line:

```
border1: ip prefix-list private-ips seq 15 permit 192.168.0.0/16 ge 16
border2: ip prefix-list private-ips seq 15 permit 192.168.0.0/16
```

The definition in `border2`'s config has accidentally omitted the `ge 16` clause and so only applies to /16 addresses.

### Example 2: Announce your own addresses out-bound


The out-bound route policies of our border routers, named `to_outside`, should allow all addresses that our network owns to be advertised.  These are addresses in the ranges 1.2.3.0/24:24-32 and 1.2.4.0/24:24-32.  We can use `searchRoutePolicies` to validate this property in the same way we did above, by asking it to find a violation of the property.  Specifically, we ask:  *Is there an announcement for an address that we own that is denied by `to_outside`?*

In [4]:
ownedSpace = ["1.2.3.0/24:24-32", "1.2.4.0/24:24-32"]
inRoutes2 = BgpRouteConstraints(prefix=ownedSpace)
bfq.searchRoutePolicies(policies="to_outside", inputConstraints=inRoutes2, action="deny").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference


Since there are no results, this implies that no such announcement exists, and therefore that our policy holds, for both `border1` and `border2`.  Hurray!

### Example 3: Announce your customers' routes out-bound

Next we will check that the `to_outside` route policies are properly announcing our customers' routes.  These are identified by announcements that are tagged with a community that has AS number 133.

First let's see how to test this property using `testRoutePolicies`.

In [5]:
inRoute3 = BgpRoute(network="1.0.0.0/8", communities=["133:10"], originatorIp="5.5.5.5", originType="egp", protocol="bgp")
bfq.testRoutePolicies(policies="to_outside", direction="out", inputRoutes=[inRoute3]).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border1,to_outside,"BgpRoute(network='1.0.0.0/8', originatorIp='5.5.5.5', originType='egp', protocol='bgp', asPath=[], communities=['133:10'], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='1.0.0.0/8', originatorIp='5.5.5.5', originType='egp', protocol='bgp', asPath=[], communities=['133:10'], localPreference=0, metric=0, sourceProtocol=None)",BgpRouteDiffs(diffs=[])
1,border2,to_outside,"BgpRoute(network='1.0.0.0/8', originatorIp='5.5.5.5', originType='egp', protocol='bgp', asPath=[], communities=['133:10'], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='1.0.0.0/8', originatorIp='5.5.5.5', originType='egp', protocol='bgp', asPath=[], communities=['133:10'], localPreference=0, metric=0, sourceProtocol=None)",BgpRouteDiffs(diffs=[])


We have tested the behavior of the `to_outside` route policies on a particular announcement, which advertises the network 1.0.0.0/8 and is tagged with the community 133:10.  The output shows that both routers permit the announcement, as desired.  However, we can use `searchRoutePolicies` to go further and check that *all* customer routes are permitted.

In [6]:
customerCommunities = "/^133:.*/"
inRoutes3 = BgpRouteConstraints(communities=customerCommunities)
bfq.searchRoutePolicies(policies="to_outside", inputConstraints=inRoutes3, action="deny").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference


Instead of specifying the set of input announcements based on their prefix, as we have done in earlier uses of `searchRoutePolicies`, above we specify them based on their communities.  In particular we are interested in the set of announcements that are tagged with some community matching the regular expression "133:.*", using the same syntax that Batfish uses for regular-expression [specifiers](https://github.com/batfish/batfish/blob/master/questions/Parameters.md#set-of-enums-or-names). We then ask `searchRoutePolicies` for announcements that are denied.  There are no answers, so we can be sure that all of our customer routes are being permitted, by both routers.

### Example 4: Filter based on origin in-bound


Last, we want to make sure that our `from_outside` route policies only accept announcements that originate at AS 44.  Further, accepted announcements should be tagged with the community 20:30.  We can use `searchRoutePolicies` to check both fo these properties.  Checking the first property is a variation on what we've done earlier.  Specifically, we will ask "Is there a route announcement whose AS-path does *not* end in 44 but is permitted?"

In [7]:
origin = "!/( |^)44$/"
inRoutes2 = BgpRouteConstraints(asPath=origin)
bfq.searchRoutePolicies(policies="from_outside", inputConstraints=inRoutes2, action="permit").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border1,from_outside,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=['20:30'], localPreference=0, metric=0, sourceProtocol=None)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[20:30]')])"


Since there are no results for `border2` we can be sure that it is performing origin filtering properly:  only announcements that originate from AS 44 are permitted.  However, the result above for `border1` shows that it can violate this property.  Inspecting the configurations, we see that both route policies define an `allowed-origins` AS-path access list, but the configuration for `border1` accidentally omits the corresponding `match` statement in its `from_outside` route policy. 

Finally, let's make sure that our `from_outside` route policies tag each permitted route with the community 20:30.  To check this property, we will leverage the ability of `searchRoutePolicies` not only to search for particular *input* announcements, but also to search for particular *output* announcements.  In this case, we will ask:  Is there a permitted route whose output announcement is not tagged with community 20:30?

In [8]:
outRoutes4 = BgpRouteConstraints(communities="!20:30")
bfq.searchRoutePolicies(policies="from_outside", action="permit", outputConstraints=outRoutes4).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border2,from_outside,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[{'asns': [44], 'confederation': False}], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[{'asns': [44], 'confederation': False}], communities=['2:30'], localPreference=0, metric=0, sourceProtocol=None)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[2:30]')])"


There are no results for `border1`'s `to_outside` policy, so it is properly tagging permitted routes.  However, Batfish reports a violation for `border2`:  its route policy tags the specified route announcement with the community 2:30.  A comparison of the two configurations identifies the problem, a typo in the `set community` command:

```
border1: set community 20:30
border2: set community 2:30
```

### Current Status

The `testRoutePolicies` question supports all of the route policies that are parsed by Batfish.  The `searchRoutePolicies` question currently supports Cisco IOS routers and the following capabilities:  matching on prefixes, communities, and AS paths; setting and deleting communities; and setting the metric and local preference.  In the near future we plan to add support for other features, notably AS prepending, as well as for other configuration formats.

### Summary


In this notebook we showed you two ways to validate that your route policies meet your intent.  The `testRoutePolicies` question allows you to easily test the behavior of a route policy offline, without access to the live network.  The `searchRoutePolicies` question allows you to search for violations of intent, thereby identifying concrete errors if they exist and providing strong correctness guarantees if not.